# 태스크 2: Retrieve API와 함께 Amazon Bedrock 지식 기반을 사용하여 Q&A 애플리케이션 구축

이 태스크에서는 **Retrieve** API와 함께 Amazon Bedrock 지식 기반을 사용하여 Q&A 애플리케이션을 구축합니다. 여기에서는 지식 기반을 쿼리하여 유사성 검색을 기반으로 원하는 수의 문서 청크를 가져옵니다. 그런 다음 관련 문서로 프롬프트를 보강하고 Amazon Nova Lite에 입력 역할을 하는 쿼리를 수행하여 응답을 생성합니다.

지식 기반을 활용하면 Amazon Bedrock의 파운데이션 모델(FM)을 검색 증강 생성(RAG)을 위한 회사 데이터에 안전하게 연결할 수 있습니다. 추가 데이터에 액세스하면 FM을 지속적으로 재훈련시키지 않아도 모델이 더 연관성 높고 상황에 맞는 정확한 응답을 생성할 수 있습니다. 지식 기반에서 검색된 모든 정보는 투명성을 개선하고 할루시네이션을 최소화하기 위해 소스 속성과 함께 제공됩니다.

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i> **자세히 알아보기:** 콘솔을 사용하여 지식 기반을 생성하는 방법에 대한 자세한 내용은 **[Amazon Bedrock 지식 기반](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)** 을 참조하십시오.

### 시나리오

검색 증강 생성(RAG) 패턴을 사용하여 솔루션을 구현합니다. RAG는 언어 모델 외부에서 데이터를 검색하고, 검색된 데이터를 컨텍스트에 추가하여 프롬프트를 증강합니다. 여기에서는 실습 프로비저닝의 일부로 생성된 지식 기반에 대한 RAG를 효과적으로 수행해 보겠습니다.
    
이 노트북에서는 다음을 수행합니다.

- AnyCompany의 financial 10k 보고서(합성하여 생성된 데이터세트)를 텍스트 말뭉치로 사용하여 Q&A를 수행합니다. 이 데이터는 실습 프로비저닝 과정에서 기존에 Amazon Bedrock 지식 기반에 수집되었습니다.
- 이 실습 환경을 위해 만든 기존 지식 기반의 지식 기반 ID를 사용합니다.
- Amazon Bedrock *Retrive API*와 LangChain 검색을 모두 사용하여 지식 기반에서 문서를 검색하고, 이를 컨텍스트로 추가하여 사용자 쿼리에 응답할 수 있도록 합니다.

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **주의:** **Run** 메뉴에서 **Run All Cells** 옵션을 사용하는 것보다 각 코드 셀을 개별적으로 실행하는 것이 좋습니다. 모든 셀을 한꺼번에 같이 실행하면 커널 충돌 또는 재시작 같은 예기치 않은 동작이 발생할 수 있습니다. 셀을 하나씩 실행하면 실행 흐름을 더욱 잘 제어하고, 잠재적인 오류를 조기에 발견하며, 의도한 대로 코드가 실행되도록 할 수 있습니다.

### 태스크 2.1: 환경 설정

이 태스크에서는 Amazon Bedrock 클라이언트를 시작하고 다음 작업을 수행합니다.

- 지식 기반 ID를 확인합니다.
- 필요한 라이브러리를 가져오고, 필요한 클라이언트를 설정합니다.

#### 태스크 2.1.1: 지식 기반 ID 확인

이 노트북을 실행하려면 지식 기반 ID를 확인한 후 **kb_id** 변수에 할당하고, 필요한 패키지를 설치해야 합니다.

1. 다음의 코드 셀을 실행하여 Amazon Bedrock에 있는 기존 지식 기반의 ID를 확인합니다.

In [1]:
import boto3
import botocore

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

Knowledge Base ID: V8BXZONLNU


#### 태스크 2.1.2: Amazon Bedrock 클라이언트 시작

2. 다음의 코드 셀을 실행하여 환경을 설정하는 데 필요한 라이브러리를 가져옵니다.

In [2]:
import boto3
from botocore.client import Config
import pprint
import json

pp = pprint.PrettyPrinter(indent=2)

session = boto3.session.Session()
region = session.region_name

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)

## 파트 1: Amazon Bedrock의 파운데이션 모델과 함께 *Retrieve* API 사용

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 이 부분에는 *amazon.nova-lite-v1:0* 모델을 사용합니다.

### 태스크 2.2: Amazon Bedrock의 파운데이션 모델과 함께 Retrieve API 사용

이 태스크에서는 Amazon Bedrock의 지식 기반에서 제공하는 **Retrive** API를 호출하는 검색 함수를 정의합니다. 이 함수는 사용자 쿼리를 임베딩으로 변환하고, 지식 기반을 검색하며, 관련 있는 결과를 반환하므로 시맨틱 검색 결과를 기반으로 사용자 지정 워크플로를 구축할 수 있는 제어 기능이 향상됩니다.

**Retrieve** API에는 **검색된 텍스트 청크**, 소스 데이터의 **위치 유형**과 **URI**, 검색의 관련성 **점수**가 포함됩니다. **HYBRID** 또는 **SEMANTIC**을 사용할 수 있는 옵션을 제공하는 **retrievalConfiguration**의 **overrideSearchType** 옵션을 사용할 수도 있습니다.

가장 관련성이 높은 결과를 제공하는 데 적합한 전략이 기본적으로 선택됩니다. 하이브리드 또는 시맨틱 검색을 사용하도록 기본 옵션을 재정의하려면 값을 **HYBRID/SEMANTIC**으로 설정할 수 있습니다.

<!-- ![retrieveAPI](./images/retrieveAPI.png) -->
<img src="images/retrieveAPI.png" width=50% height=20% />

**이미지 설명: 위 다이어그램은 실습 환경에 맞게 사용자 지정된 RAG 워크플로를 보여줍니다.**

3. 다음의 코드 셀을 실행하여 **Retrieve** API를 호출하는 **retrieve** 함수를 정의합니다.

In [3]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

#### 태스크 2.2.1: 초기화된 LLM에서 응답을 쿼리하기 전에 지식 기반 ID 초기화

이 태스크에서는 **Retrieve** API를 호출하고 **지식 기반 ID**, **결과 수**, **쿼리**를 파라미터로 전달합니다. 

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 반환된 각 텍스트 청크의 관련 점수를 볼 수 있습니다. 이 점수는 쿼리와 얼마나 밀접하게 일치하는지에 대한 상관관계를 보여줍니다.

4. 다음의 코드 셀을 실행하여 **지식 기반 ID**, **결과 수**, **쿼리**를 파라미터로 전달함으로써 **Retrieve** API를 호출합니다.

In [6]:
query = "What was the total operating lease liabilities and total sublease income of the AnyCompany as of December 31, 2022?"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

[ { 'content': { 'text': 'The lease payments are structured in a way that '
                         'provides AnyCompany with a profit or loss on the '
                         'sale of the underlying asset. As of December 31, '
                         '2021, AnyCompany has entered into sales-type leases '
                         'with a total lease receivable of $20 million and a '
                         'weighted-average remaining lease term of 3 '
                         'years.      The following table summarizes '
                         "AnyCompany's lease portfolio as of December 31, "
                         '2021:      | Lease Type | Total Lease Liability | '
                         'Weighted-Average Remaining Lease Term | | --- | --- '
                         '| --- | | Operating Leases | $50 million | 5 years | '
                         '| Financing Leases | $30 million | 7 years | | '
                         'Sales-Type Leases | $20 million | 3 years |      In '

In [5]:
query = "AnyCompany의 2022년 12월 31일 기준 총 운용리스 부채와 총 전대차 수익은 얼마였습니까?"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

[ { 'content': { 'text': '----- | | Earnings Per Share | $3.20 | $3.00 | | '
                         'Shares Outstanding | 375,000 | 375,000 |           '
                         'In 2021, AnyCompany Financial reported total revenue '
                         'of $5,000 million, an increase of $500 million or '
                         '11.1% compared to $4,500 million in 2020. This '
                         'growth was driven by strong performance in the '
                         "company's core business segments, as well as the "
                         'acquisition of several smaller companies.      Cost '
                         'of revenue for 2021 was $2,000 million, an increase '
                         'of $200 million or 11.1% compared to $1,800 million '
                         'in 2020. This increase was primarily due to higher '
                         'raw material costs and increased labor costs '
                         "associated with the company's growth.   

#### 태스크 2.2.2: **Retrieve** API 응답에서 텍스트 청크 추출

이 태스크에서는 **Retrieve** API 응답에서 텍스트 청크를 추출합니다.

5. 다음의 코드 셀 2개를 실행하여 검색 결과에서 컨텍스트를 가져와 인쇄합니다.

In [7]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [8]:
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

[ 'The lease payments are structured in a way that provides AnyCompany with a '
  'profit or loss on the sale of the underlying asset. As of December 31, '
  '2021, AnyCompany has entered into sales-type leases with a total lease '
  'receivable of $20 million and a weighted-average remaining lease term of 3 '
  "years.      The following table summarizes AnyCompany's lease portfolio as "
  'of December 31, 2021:      | Lease Type | Total Lease Liability | '
  'Weighted-Average Remaining Lease Term | | --- | --- | --- | | Operating '
  'Leases | $50 million | 5 years | | Financing Leases | $30 million | 7 years '
  '| | Sales-Type Leases | $20 million | 3 years |      In addition to the '
  'above, it is important to note that AnyCompany has adopted ASC 842, Leases, '
  'as of January 1, 2022. This new standard requires lessees to recognize a '
  'lease liability and a right-of-use (ROU) asset for all leases, with the '
  'exception of short-term leases. The new standard also requires 

#### 태스크 2.2.3: 모델에 대한 특정 프롬프트를 사용하여 맞춤형 응답 생성 

이 태스크에서는 모델에 대한 특정 프롬프트를 사용하여 재무 자문 AI 시스템 역할을 수행하도록 할 수 있습니다. 이러한 시스템은 가능한 한 사실에 기반한 정보와 통계 정보를 사용하여 질문에 대한 답변을 제공합니다. 참조할 모델에 대한 프롬프트에 있는 **{contexts}** 의 일환으로, 이전 태스크의 **Retrieve** API 응답을 사용자 **쿼리**와 함께 제공하십시오.

6. 다음의 코드 셀을 실행하여 특정 프롬프트를 사용해 모델이 재무 자문 AI 시스템 역할을 수행하도록 합니다.

In [9]:
prompt = f"""
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

#### 태스크 2.2.4: Amazon Bedrock에서 파운데이션 모델 간접 호출

이 태스크에서는 Amazon Bedrock의 **amazon.nova-lite-v1:0** 파운데이션 모델을 사용합니다.

7. 다음 2개의 코드 셀을 실행하여 Amazon Bedrock에서 **amazon.nova-lite-v1:0** 파운데이션 모델을 간접적으로 호출합니다. 컨텍스트와 쿼리를 둘 다 모델에 전달하게 됩니다.

In [10]:
# payload with model parameters
messages = [{
    "role": "user",
    "content": [{"text": prompt}]
}]

# Create the proper Nova Lite payload
nova_payload = {
    "schemaVersion": "messages-v1",
    "messages": messages,
    "inferenceConfig": {
        "maxTokens": 512,
        "temperature": 0.5,
        "topP": 0.9,
        "topK": 20
    }
}

In [11]:
modelId = 'amazon.nova-lite-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock_client.invoke_model(
    body=json.dumps(nova_payload),
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

# Parse and extract the response
response_body = json.loads(response.get('body').read())

# Extract just the text from the response
response_text = ''
if 'output' in response_body and 'message' in response_body['output']:
    message_content = response_body['output']['message']['content']
    if message_content and isinstance(message_content, list):
        response_text = message_content[0].get('text', '')

# Print the response text
print(response_text)

The context provided does not include specific information about the total sublease income for AnyCompany as of December 31, 2022. However, it does mention that AnyCompany has operating leases with a total lease liability of $50 million as of December 31, 2021. Since there is no update provided for the year 2022, we can only confirm the operating lease liability as of December 31, 2021. 

As for the total sublease income, the context does not provide any data points, so I cannot provide a specific number. 

Therefore, as of December 31, 2021:
- Total Operating Lease Liabilities: $50 million
- Total Sublease Income: Not available in the provided context


## 파트 2: LangChain 통합

### 태스크 2.3: LangChain 통합

이 태스크에서는 LangChain의 **AmazonKnowledgeBasesRetriever** 클래스를 사용하여 Q&A 애플리케이션을 구축합니다. 지식 기반을 쿼리하여 유사성 검색을 기반으로 문서 청크를 원하는 수만큼 가져옵니다. 그런 다음 LangChain 체인과 통합하여 질문에 답하기 위해 문서 청크와 쿼리를 LLM(**Amazon Nova Lite**)에 전달합니다.

#### 태스크 2.3.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

8. 다음의 코드 셀을 실행하여 환경을 설정하는 데 필요한 패키지를 가져옵니다.

In [12]:
import langchain
from langchain_aws import ChatBedrock
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever

llm = ChatBedrock(model_id=modelId, 
                  client=bedrock_client)

#### 태스크 2.3.2: Retrieve API를 호출하는 AmazonKnowledgeBasesRetriever 객체 생성

이 태스크에서는 LangChain에서 Amazon Bedrock 지식 기반에서 제공되는 **Retrieve** API를 호출하는 **AmazonKnowledgeBasesRetriever** 객체를 생성합니다. 이렇게 하면 사용자 쿼리가 임베딩으로 변환되고, 지식 기반을 검색하며, 관련 있는 결과가 반환되므로 시맨틱 검색 결과를 기반으로 사용자 지정 워크플로를 구축할 수 있는 제어 기능이 향상됩니다.

9. 다음의 코드 셀을 실행하여 *AmazonKnowledgeBasesRetriever* 객체를 생성합니다.

In [13]:
query = "What was the total operating lease liabilities and total sublease income of the AnyCompany as of December 31, 2022?"
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 4,
                           'overrideSearchType': "SEMANTIC", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )
docs = retriever.invoke(
        input=query
    )
for doc in docs:
    print(doc.page_content)
    print("------")

/tmp/ipykernel_2584/3114278716.py:2: LangChainDeprecationWarning: The class `AmazonKnowledgeBasesRetriever` was deprecated in LangChain 0.3.16 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import AmazonKnowledgeBasesRetriever``.
  retriever = AmazonKnowledgeBasesRetriever(


The lease payments are structured in a way that provides AnyCompany with a profit or loss on the sale of the underlying asset. As of December 31, 2021, AnyCompany has entered into sales-type leases with a total lease receivable of $20 million and a weighted-average remaining lease term of 3 years.      The following table summarizes AnyCompany's lease portfolio as of December 31, 2021:      | Lease Type | Total Lease Liability | Weighted-Average Remaining Lease Term | | --- | --- | --- | | Operating Leases | $50 million | 5 years | | Financing Leases | $30 million | 7 years | | Sales-Type Leases | $20 million | 3 years |      In addition to the above, it is important to note that AnyCompany has adopted ASC 842, Leases, as of January 1, 2022. This new standard requires lessees to recognize a lease liability and a right-of-use (ROU) asset for all leases, with the exception of short-term leases. The new standard also requires lessees to classify leases as either finance or operating lease

#### 태스크 2.3.3: 모델에 대한 특정 프롬프트를 사용하여 맞춤형 응답 가져오기

이 태스크에서는 모델에 대한 특정 프롬프트를 사용하여 재무 자문 AI 시스템 역할을 수행하도록 할 수 있습니다. 이러한 시스템은 가능한 한 사실에 기반한 정보와 통계 정보를 사용하여 질문에 대한 답변을 제공합니다. 참조할 모델에 대한 프롬프트에 있는 **{context}** 의 일환으로, 위의 **Retrieve** API 응답을 사용자 **쿼리**와 함께 제공하십시오.

10. 다음의 코드 셀을 실행하여 특정 프롬프트를 사용해 모델이 재무 자문 AI 시스템 역할을 수행하도록 합니다.

In [14]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
nova_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

#### 태스크 2.3.4: 검색기와 LLM을 검색 체인과 통합하여 Q&A 애플리케이션 구축

이 태스크에서는 LangChain Expression Language(LCEL)를 사용해 검색기 및 LLM을 통합하여 Q&A 애플리케이션을 구축합니다.

11. 다음의 셀을 실행하여 retriver.invoke를 사용해 검색한 문서와 검색기 및 LLM을 통합합니다. 결과 인쇄:

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs): #concatenate the text from the page_content field in the output from retriever.invoke
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | nova_prompt
    | llm
    | StrOutputParser()
)

response=chain.invoke(query)
print(response)

I do not have information regarding the total sublease income of AnyCompany as of December 31, 2022. However, based on the provided context, the total operating lease liabilities of AnyCompany as of December 31, 2021, were $50 million. Given that no updates are provided for the year 2022, it is not possible to provide a specific figure for the total operating lease liabilities as of December 31, 2022.


<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **태스크 완료:** 이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 태스크 3을 계속 진행합니다.

## 파트 3: LangGraph 통합

### 태스크 2.3: LangGraph 통합

이 태스크에서는 LangChain의 **AmazonKnowledgeBasesRetriever** 클래스를 사용하여 Q&A 애플리케이션을 구축합니다. 지식 기반을 쿼리하여 유사성 검색을 기반으로 문서 청크를 원하는 수만큼 가져옵니다. 그런 다음 LangChain 체인과 통합하여 질문에 답하기 위해 문서 청크와 쿼리를 LLM(**Amazon Nova Lite**)에 전달합니다.

In [24]:
%pip install langgraph

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.82
    Uninstalling langchain-core-0.3.82:m━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [langchain-core]
      Successfully uninstalled langchain-core-0.3.82━━━━━━━━━━ 2/6 [langchain-core]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [langgraph]/6 [langgraph]checkpoint]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.2.7 which is incompatible.
langchain-aws 0.2.35 requires langchain-core<0.4.0,>=0.3.76, but you have langchain-core 1.2.7 which is incompatible.
langchain-openai 0.3.35 requires langchain-core<1.0.0,>=0.3.78, but you have langchain-core 1.2.7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from typing import TypedDict, List
from langchain_core.documents import Document

class GraphState(TypedDict):
    question: str
    docs: List[Document]
    context: str
    answer: str

In [17]:
from langchain_aws import ChatBedrock
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever

llm = ChatBedrock(
    model_id=modelId,
    client=bedrock_client
)

retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id=kb_id,
    retrieval_config={
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            "overrideSearchType": "SEMANTIC",
        }
    }
)

In [18]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:
"""

nova_prompt = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)


In [19]:
def retrieve_node(state: GraphState) -> GraphState:
    docs = retriever.invoke(state["question"])
    return {
        **state,
        "docs": docs,
    }

In [20]:
def format_docs_node(state: GraphState) -> GraphState:
    context = "\n\n".join(doc.page_content for doc in state["docs"])
    return {
        **state,
        "context": context,
    }

In [21]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

def llm_node(state: GraphState) -> GraphState:
    prompt = nova_prompt.format(
        context=state["context"],
        question=state["question"]
    )

    response = llm.invoke(prompt)
    answer = output_parser.invoke(response)

    return {
        **state,
        "answer": answer,
    }


In [22]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

def llm_node(state: GraphState) -> GraphState:
    prompt = nova_prompt.format(
        context=state["context"],
        question=state["question"]
    )

    response = llm.invoke(prompt)
    answer = output_parser.invoke(response)

    return {
        **state,
        "answer": answer,
    }


In [25]:
from langgraph.graph import StateGraph, END

graph = StateGraph(GraphState)

graph.add_node("retrieve", retrieve_node)
graph.add_node("format_context", format_docs_node)
graph.add_node("llm", llm_node)

graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "format_context")
graph.add_edge("format_context", "llm")
graph.add_edge("llm", END)

app = graph.compile()


In [26]:
query = "What was the total operating lease liabilities and total sublease income of the AnyCompany as of December 31, 2022?"

result = app.invoke({
    "question": query
})

print(result["answer"])


The context provided does not include specific information about the total sublease income for AnyCompany as of December 31, 2022. However, based on the information provided, the total operating lease liabilities as of December 31, 2021, were $50 million. To answer the question about the total operating lease liabilities as of December 31, 2022, additional information would be required. 

For the total sublease income as of December 31, 2022, the context does not provide this information, so I cannot provide a specific number.


##### LangChain

```python
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | nova_prompt
    | llm
    | StrOutputParser()
)
```

##### LangGraph

```text
question
  ↓
retrieve_node
  ↓
format_docs_node
  ↓
llm_node
  ↓
answer
```

---

➡️ 기능은 동일,

➡️ 확장성·제어력은 LangGraph가 압도적으로 큼